# Import Library

In [1]:
from pandas_datareader import data as DATA
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import SimpleRNN, LSTM
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
import itertools

Using TensorFlow backend.


# Import Data

### 1. Stock Price Data
#### Here we first load all the stock price in our portfolio. This is the raw data of our input and it will later be used in calculation the performance of model and benchmarks.

In [2]:
lst={'AAPL','AMT','AMZN','BA','JNJ','JPM','LIN','NEE','PG','T','XOM'}
df_stock=pd.DataFrame(data=None)
for stock in lst:
    data=pd.read_excel('./FRE7773Data/'+stock+'.xlsx',header=None,index_col=None,usecols=[0,1])[8:]
    data.rename(columns={0:'date',1:stock},inplace=True)
    data['date']=pd.to_datetime(data['date'])
    data.set_index('date',inplace=True)
    df_stock=pd.concat([df_stock,data], axis=1)
    # df=df.sort_values(by = 'date')
df_stock=df_stock.sort_index()
df_stock=df_stock.astype(float)
df_stock.head()

,JPM,T,PG,AMZN,XOM,JNJ,NEE,BA,AMT,LIN,AAPL
date,,,,,,,,,,,
2008-12-31,31.53,28.50,61.82,51.28,79.83,59.83,50.33,42.67,29.32,59.36,12.1929
2009-01-02,31.35,29.42,62.80,54.36,81.64,60.65,51.66,45.25,30.19,62.43,12.9643
2009-01-05,29.25,28.43,62.35,54.06,81.63,60.05,51.94,46.17,30.07,63.87,13.5114
2009-01-06,29.88,28.30,62.17,57.36,80.30,59.69,51.21,46.31,30.23,66.41,13.2886
2009-01-07,28.09,27.21,61.08,56.20,78.25,59.13,50.65,44.76,29.06,62.58,13.0014


### 2. Preprocess Input Data
#### Input Data is preprocessed in another code file which gets the covariance and variance from raw stock prices of the assets. According to the result of rolling backtest and grid research, the time window of covariance and variance (TW1)  is set to 60 days.

In [5]:
TW1=60
# df=pd.read_csv('../tw='+str(TW1),index_col = 0)#Change path when this doesn't match yours
df=pd.read_csv('./FRE7773Data/tw='+str(TW1)+'.csv',index_col = 0)
df.dropna(inplace=True)
df.shape

(2686, 77)

#### Input Matrix is (2686,77), which means there are 77 features and 2686 samples.

### 3. Output Data Calculated with HRP
#### Output data is calculated using the returns in X future days using HRP algorithm. The calculation can be found in another code file. Here we load the result of calculation as our output. X is also decided by rolling backtest and grid research.

In [7]:
TW2=10
# y =pd.read_csv('../Y_HRP_v2/Y_HRP_tw='+str(TW2)+'.csv',index_col = 0)
y =pd.read_csv('./Y_HRP_v2/Y_HRP_tw='+str(TW2)+'.csv',index_col = 0)
y.shape

(2736, 11)

### 4. Treasury Yield 10 Years (^TNX) 
#### Take this as the proxy of risk free rate to calculate Sharpe Ratio later.


In [8]:
rf_data = DATA.DataReader('^TNX', 'yahoo', '12/31/2008','11/30/2019')
rf_data = rf_data.dropna()

# Process Input Data 

### 1. Split dataset
#### Here we still use rolling time window to access the performance of our model. The length of time window is set to 3 years and the step is set to 1 year. As validation data are used to tune hyperparameters of the model, it's not needed here. Therefore, validation part is included in training data to make the best use of data information. So the split ratio is 9:1 for training to testing.

In [9]:
def SplitData(SW_i,SW_step,SW_len,df,y):
    # SW_i : index of time window
    # SW_step: the step between the adjacent time windows
    # SW_len: the length of time window
    # df,y: input and outpu
    ind_train_s=SW_i*SW_step
    ind_test_s=int(SW_i*SW_step+SW_len*0.9)
    ind_test_end=SW_i*SW_step+SW_len
    train_data = df.iloc[ind_train_s:ind_test_s,:] 
    test_data = df.iloc[ind_test_s:ind_test_end,:]
    y_train = y.iloc[ind_train_s:ind_test_s,:]
    y_test = y.iloc[ind_test_s:ind_test_end,:]
    train_data = train_data.values.reshape(-1,77)
    test_data = test_data.values.reshape(-1,77)
    return (train_data,test_data,y_train,y_test)

### 2. Normalize data
#### The input data is acquired from the stock price without normalization while stock prices vary from each other a lot. Here we normalize it using only the training data. This means the test data is normalized with the information of training data. As the time length of rolling window is set to 3 years, there is no need to cut the normalization to several periods. 

In [10]:
def Normalization(train_data,test_data):
    scaler = MinMaxScaler()
    scaler.fit(train_data)
    train_data= scaler.transform(train_data)
    test_data = scaler.transform(test_data)
    return(train_data,test_data)

### 3. Shape data for LSTM 
#### LSTM is trained with a 3-dimension input while our input is 2-dimension. Here we need to shape data according to the number of time steps to be considered in one training. Here the number is set to 60.

In [11]:
def build_timeseries(mat, r, TIME_STEPS):
    # total number of time-series samples would be len(mat) - TIME_STEPS
    dim_0 = mat.shape[0] - TIME_STEPS
    dim_1 = mat.shape[1]
    x = np.zeros((dim_0, TIME_STEPS, dim_1))
#     y = np.zeros((dim_0, r.shape[1]))
    
    for i in range(dim_0):
        x[i] = mat[i:TIME_STEPS+i]
#         y[i] = r[i:i+1]
    y = pd.DataFrame(data=r.iloc[:dim_0,:])
    print("length of time-series input/output:",x.shape,y.shape)
    return x, y

def ShapeLSTM(train_data,y_train,test_data,y_test,TS):
    X_train, Y_train = build_timeseries(train_data, y_train, TS)
    #X_train, Y_train = trim_dataset(X_train, BATCH_SIZE), trim_dataset(Y_train, BATCH_SIZE)
    X_test, Y_test = build_timeseries(test_data, y_test, TS)
    #X_test, Y_test = trim_dataset(X_test, BATCH_SIZE), trim_dataset(Y_test, BATCH_SIZE)
    return(X_train,Y_train,X_test,Y_test)

# Build and Train Model
#### According to previous result, LSTM performs better than ANN. 

In [12]:
def BuildNTrainModel(TS,X_train,Y_train):
    model = Sequential()
    # model.add(LSTM(32, batch_input_shape=(10, x_train.shape[1], x_train.shape[2]), stateful=True))
    model.add(LSTM(32, input_shape=(TS, X_train.shape[2]), dropout=0.2, activation='relu'))
    #model.add(Dense(32,activation='relu'))
    model.add(Dense(11, activation='linear'))
    model.summary()
    model.compile(loss='mean_squared_error',optimizer='adam')
    #Train
    callback = EarlyStopping(monitor='loss', patience=3, mode='auto', min_delta=0.0001)
    history = model.fit(X_train, Y_train, epochs=20, batch_size=20,callbacks=[callback])
    return model

# Predict and Calculate Metrics

In [13]:
def CalCumReturn(data, tw):
    data = data + 1
    Cum = data.to_frame(name="Cum_return")
    i = np.linspace(0, tw*(len(data)//tw), num=1+len(data)//tw, endpoint=False)
    
    Cum = Cum.iloc[i]
    Cum = Cum.cumprod()
    return Cum

def Sample(data,tw):
    data = data.to_frame(name="return_tw")
    i = np.linspace(0, tw*(len(data)//tw), num=1+len(data)//tw, endpoint=False)
    rebalance_r = data.iloc[i]
    return rebalance_r

def CalMetric(data,tw):
    cum=CalCumReturn(data,tw)
    df=float(1.0/(len(cum)*tw)*252)
    Ann_R=(cum.iloc[-1].values-1)*df
    Vol=Sample(data,tw).std().values
    Ann_Vol=Vol*((252/tw)**0.5)
    return(Ann_R,Ann_Vol)

def Sharpe_ratio(r, tw, rf_data, days=252):
#     Calcualte annual annual Sharpe ratio
#     INPUT: part of return_hrp(rebalance_r), not CumHRP
    annual_r, annual_vol = CalMetric(r,tw)[0], CalMetric(r,tw)[1]
    
    rf = rf_data.loc[r.index[0]:r.index[-1], ['Close']]
    rf = rf.mean()*0.01
    sharpe_ratio = float((annual_r - rf) / annual_vol)
    return sharpe_ratio

def CalReturn(data, tw):
# #     Time interval is tw, eg: pct_change of the 21st and 1st
#     r = data.pct_change(tw)[tw:]
#     r.index = data.index[:-tw]
# #     r.columns = "R_"+r.columns
#     Time interval is tw, eg: pct_change of the 20th and 1st
    r = data.pct_change(tw-1)[tw-1:]
    r.index = data.index[:-(tw-1)]
#     r.columns = "R_"+r.columns
    return r

In [14]:
def PredictNCal(model,X_test,Y_test,tw):
    y_pred=model.predict(X_test,batch_size=20)
    y_pred_df=pd.DataFrame(data=y_pred,index=Y_test.index,columns=Y_test.columns)
    y_normalized=y_pred_df.div(y_pred_df.sum(axis=1), axis=0)
    #
    df_return=CalReturn(df_stock,tw)
    df_return=df_return.astype(float)
    df_return=df_return.reindex(y_normalized.columns, axis=1)
    df_return_period = df_return[df_return.index.isin(y_normalized.index)]
    #
    meanweighted=df_return_period.mean(axis=1)
    model_result=pd.DataFrame(df_return_period.values*y_normalized.values,columns=y_normalized.columns,index=meanweighted.index).sum(axis=1)
    hrp_result=pd.DataFrame(df_return_period.values*Y_test.values,columns=Y_test.columns,index=meanweighted.index).sum(axis=1)
    #
    Result=np.zeros((3,3))
    Result[0]=[CalMetric(meanweighted,tw)[0], CalMetric(meanweighted,tw)[1], Sharpe_ratio(meanweighted, tw, rf_data, days=252)]
    Result[1]=[CalMetric(model_result,tw)[0], CalMetric(model_result,tw)[1], Sharpe_ratio(model_result, tw, rf_data, days=252)]
    Result[2]=[CalMetric(hrp_result,tw)[0], CalMetric(hrp_result,tw)[1], Sharpe_ratio(hrp_result, tw, rf_data, days=252)]
    Result_df=pd.DataFrame(Result,columns=['AnnualReturn','Volatility','SharpeRatio'],index=['Mean','Model','HRP'])
    Result_df['AnnualReturn']=pd.Series(["{0:.2f}%".format(val * 100) for val in Result_df['AnnualReturn']], index = Result_df.index)
    return Result_df

# Run Rolling Time Windows

In [15]:
SW_Result=pd.DataFrame(None)
SW_step=252
SW_len=756
for SW_i in range(0,7):
    TS=60
    tw=TW2
    train_data,test_data,y_train,y_test = SplitData(SW_i,SW_step,SW_len,df,y)
    train_data,test_data = Normalization(train_data,test_data)
    X_train,Y_train,X_test,Y_test = ShapeLSTM(train_data,y_train,test_data,y_test,TS)
    model = BuildNTrainModel(TS,X_train,Y_train)
    Result_df = PredictNCal(model,X_test,Y_test,tw)
    SW_Result=pd.concat([SW_Result,Result_df.T.unstack(level=-1)],axis=1)
    SW_Result.rename(columns={0:'SW'+str(SW_i)},inplace=True)

length of time-series input/output: (620, 60, 77) (620, 11)
length of time-series input/output: (16, 60, 77) (16, 11)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                14080     
_________________________________________________________________
dense_1 (Dense)              (None, 11)                363       
Total params: 14,443
Trainable params: 14,443
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
620/620 [==============================] - 7s 11ms/step - loss: 0.4929
Epoch 2/20
620/620 [==============================] - 5s 8ms/step - loss: 0.0315
Epoch 3/20
620/620 [==============================] - 5s 9ms/step - loss: 0.0221
Epoch 4/20
620/620 [==============================] - 7s 11ms/step - loss: 0.0179
Epoch 5/20
620/620 [==============================] - 6s 10ms/step - loss: 0.0155
Epoch 6/

620/620 [==============================] - 6s 10ms/step - loss: 0.0078
Epoch 7/20
620/620 [==============================] - 5s 8ms/step - loss: 0.0076
Epoch 8/20
620/620 [==============================] - 7s 11ms/step - loss: 0.0074
Epoch 9/20
620/620 [==============================] - 6s 9ms/step - loss: 0.0073
Epoch 10/20
620/620 [==============================] - 6s 10ms/step - loss: 0.0073
Epoch 11/20
620/620 [==============================] - 5s 8ms/step - loss: 0.0071
Epoch 12/20
620/620 [==============================] - 5s 8ms/step - loss: 0.0071
Epoch 13/20
620/620 [==============================] - 5s 9ms/step - loss: 0.0070
Epoch 14/20
620/620 [==============================] - 5s 8ms/step - loss: 0.0069
Epoch 15/20
620/620 [==============================] - 5s 8ms/step - loss: 0.0069
Epoch 16/20
620/620 [==============================] - 5s 8ms/step - loss: 0.0068
Epoch 17/20
620/620 [==============================] - 5s 8ms/step - loss: 0.0067
Epoch 18/20
620/620 [=======

In [16]:
SW_Result.to_csv('SW_Result.csv')

In [17]:
SW_Result

SW0        SW1         SW2        SW3       SW4  \
Mean  AnnualReturn    -27.25%    -20.50%     -21.31%    -51.81%    16.13%   
      Volatility    0.0956373   0.035482  0.00903989  0.0790736   0.21227   
      SharpeRatio    -3.04975   -6.26176    -26.5297    -6.8681  0.660701   
Model AnnualReturn     -6.28%     -2.23%     -36.66%    -63.47%    22.27%   
      Volatility    0.0709236  0.0214861  0.00300717  0.0830322  0.211738   
      SharpeRatio    -1.15645   -1.83625    -130.768   -7.94498  0.952348   
HRP   AnnualReturn    -39.41%     -6.52%     -39.30%    -37.73%    17.76%   
      Volatility     0.164357  0.0388425   0.0254195  0.0285272  0.182238   
      SharpeRatio     -2.5144   -2.12074    -16.5115   -14.1019  0.859037   

                          SW5        SW6  
Mean  AnnualReturn     21.69%     17.48%  
      Volatility    0.0502168   0.070506  
      SharpeRatio     3.99134    2.15266  
Model AnnualReturn    -12.28%   1224.69%  
      Volatility    0.0514242    2.46387  
      SharpeRatio    -2.70852    4.96127  
HRP   AnnualReturn      6.89%     17.70%  
      Volatility    0.0440473  0.0461091  
      SharpeRatio     1.19086    3.34024